#### FAISS(Facebook, AI, Similarity, Search)
- Fasebook AI Research 에서 개발한 C++ 언어 라이브러리
- 벡터 DB
- 기존의 cosine_similarity 로 전체 비교하던 부분을 벡터 DB 의 인덱스 검색으로 전환

In [1]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
    --------------------------------------- 0.3/18.9 MB ? eta -:--:--
   --------------------------- ------------ 13.1/18.9 MB 57.0 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 50.9 MB/s  0:00:00



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import numpy as np
import faiss
from sklearn.preprocessing import normalize

load_dotenv(find_dotenv())

True

In [4]:
# 아마존 음식점 리뷰
amazone_df = pd.read_csv("./data/fine_food_reviews_with_embeddings_1k.csv",index_col=0)
# "[0.~~~]" "" 제거
amazone_df['embedding'] = amazone_df.embedding.apply(eval).to_list()
amazone_df.head(2)

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...,87,"[-0.0022805905900895596, 0.0007801640313118696..."
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...,55,"[0.05709123983979225, 0.005073584616184235, -0..."


In [7]:
client = OpenAI()

# ---------- 1) 인덱스 만들기 ----------
def build_faiss_from_df_embeddings(df: pd.DataFrame, emb_col="embedding"):
    vectors = np.vstack(df[emb_col].to_numpy()).astype("float32")
    # 정규화
    vectors = normalize(vectors)

    # 차원 가져오기(1000,1536)
    d = vectors.shape[1]
    index = faiss.IndexFlatIP(d) #indexFlatIP : 명시적 ID 를 저장하지 않음
    index.add(vectors)

    id_map = df.index.to_numpy()
    return index, id_map
# ---------- 2) 검색 ----------
def search_faiss_with_existing_embeddings(
    df: pd.DataFrame,
    index,
    id_map,
    query: str,
    top_k: int = 3,
    model: str = "text-embedding-3-small"   
):
    # 사용자 입력 -> embedding
    embedding = client.embeddings.create(input=query, model=model).data[0].embedding
    q = np.array(embedding, dtype="float32").reshape(1,-1)

    # 질문 정규화 
    q= normalize(q)

    scores, idxs = index.search(q, top_k) # 08에서 fori 허자얺고 바로 찾기
    rows = id_map[idxs[0]]

    # 실제 리뷰 찾기
    result = df.loc[rows].copy()
    result['similarity'] = scores[0]

    return result.sort_values("similarity", ascending=False)

# ---------------- 사용 예 ----------------

index, id_map = build_faiss_from_df_embeddings(amazone_df,emb_col="embedding")
res = search_faiss_with_existing_embeddings(amazone_df,index,id_map,query="맛있는 콩",top_k=3)
res

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,similarity
771,B002GWHBU2,A2NUQSB42RKBFA,5,자메이카 푸른 콩,로스팅하기위한 우수한 커피 콩 우리 가족은 더 많은 양의 맛을 위해 5 파운드를 더...,Title: 자메이카 푸른 콩; Content: 로스팅하기위한 우수한 커피 콩 우리...,112,"[0.0030990454833954573, 0.03926049545407295, -...",0.520019
88,B000E3ZFDU,A1PQDL14230X6U,5,맛있는,나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍부한 맛을줍니다. 나는 법에있는 어머니...,Title: 맛있는; Content: 나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍...,181,"[0.04669349640607834, 0.03440799564123154, -0....",0.519946
460,B000RZNY5G,A2ZGTXBI8C7FYI,5,아주 맛있는,아주 좋은 것은 여전히 ​​약간 왼쪽을 가지고 있습니다. 나는 짠 맛이 마음에 든다.,Title: 아주 맛있는; Content: 아주 좋은 것은 여전히 ​​약간 왼쪽을 ...,54,"[-0.009099675342440605, -0.0311743151396513, -...",0.485047


##### 네이버 영화 리뷰

In [9]:
# 1000개의 샘플링 데이터 가져오기
train_sample = pd.read_csv("./data/train_sample.csv")

# 임베딩 데이터 가져오기
X_train = np.load("./data/X_train.npy")


In [10]:
# --- cosine을 위해 L2 정규화 ---
vectors = normalize(X_train)

# 인덱스 만들기
# 차원 가져오기(1000,1536)
d = vectors.shape[1]
index = faiss.IndexFlatIP(d) #indexFlatIP : 명시적 ID 를 저장하지 않음
index.add(vectors) 

# 사용자 입력 -> embedding
query = "영화가 재미가 없다"
model = "text-embedding-3-small"

embedding = client.embeddings.create(input=query, model=model).data[0].embedding
q = np.array(embedding, dtype="float32").reshape(1,-1)

# 질문 정규화 

q= normalize(q)
scores, idxs = index.search(q, 5)

rows = idxs[0]
scores = scores[0]

for idx, no in enumerate(rows):
    doc = train_sample.iloc[no]['document']
    label = train_sample.iloc[no]["label"]
    sim = scores[idx]
    print(f"sim={sim}, label = {label}, doc = {doc}")

sim=0.5893311500549316, label = 0, doc = 영화가 긴장감도 없고 재미도 없고 영화보면서 욕하기는 처음이네 참 류승범 연기력은 있어도 시나리오 보는 눈은 없는것 같다
sim=0.5865670442581177, label = 0, doc = 지루할뿐인영상외엔 볼게 없는 영화
sim=0.576795220375061, label = 1, doc = 이거 재미있게 본영화
sim=0.5763489603996277, label = 1, doc = 아름다운 영화라고 밖에 표현할게 없다.
sim=0.5654844045639038, label = 0, doc = 코미디 영화인데 중요한건 웃음이 없다..이런영화는 좀 만들지말았으면..
